#Model being implemented: CNN (on the raw extracted dataset)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [3]:
print("Implementing CNN from scratch using NumPy (CPU-only)")

# Load dataset
X_features = np.load("X_features.npy")
y_labels = np.load("y_labels.npy")

Implementing CNN from scratch using NumPy (CPU-only)


#Data Preprocessing

In [4]:
# Data preprocessing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_features)

# Reshape to 2D "images"
height = int(np.sqrt(X_scaled.shape[1]))
width = int(np.ceil(X_scaled.shape[1] / height))
padding = height * width - X_scaled.shape[1]
X_padded = np.pad(X_scaled, ((0, 0), (0, padding)), mode='constant')
X_reshaped = X_padded.reshape(-1, 1, height, width)  # (N, C, H, W)

# Train-test split (using raw integer labels)
X_train, X_test, y_train, y_test = train_test_split(
    X_reshaped, y_labels, test_size=0.2, random_state=42, stratify=y_labels
)

In [13]:
# Class for CNN
class ScratchCNN:
    def __init__(self, input_shape, num_classes):
        self.params = {}
        # Conv Layer 1: 3x3 kernel, 32 filters
        self.params['W1'] = np.random.randn(32, 1, 3, 3) * 0.1
        self.params['b1'] = np.zeros(32)
        # Conv Layer 2: 3x3 kernel, 64 filters
        self.params['W2'] = np.random.randn(64, 32, 3, 3) * 0.1
        self.params['b2'] = np.zeros(64)
        # FC Layer
        flattened_size = 64 * (height//4) * (width//4)
        self.params['W3'] = np.random.randn(flattened_size, 256) * 0.1
        self.params['b3'] = np.zeros(256)
        # Output Layer
        self.params['W4'] = np.random.randn(256, num_classes) * 0.1
        self.params['b4'] = np.zeros(num_classes)

    def relu(self, x):
        return np.maximum(0, x)

    def softmax(self, x):
        exps = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exps / np.sum(exps, axis=1, keepdims=True)

    def forward(self, x):
        # Conv1 -> ReLU -> MaxPool
        self.cache = {}
        self.cache['Z1'] = self._conv2d(x, self.params['W1'], self.params['b1'])
        self.cache['A1'] = self.relu(self.cache['Z1'])
        self.cache['P1'] = self._maxpool2d(self.cache['A1'])

        # Conv2 -> ReLU -> MaxPool
        self.cache['Z2'] = self._conv2d(self.cache['P1'], self.params['W2'], self.params['b2'])
        self.cache['A2'] = self.relu(self.cache['Z2'])
        self.cache['P2'] = self._maxpool2d(self.cache['A2'])

        # Flatten -> FC -> Output
        self.cache['F'] = self.cache['P2'].reshape(self.cache['P2'].shape[0], -1)
        self.cache['Z3'] = np.dot(self.cache['F'], self.params['W3']) + self.params['b3']
        self.cache['A3'] = self.relu(self.cache['Z3'])
        self.cache['Z4'] = np.dot(self.cache['A3'], self.params['W4']) + self.params['b4']
        self.cache['A4'] = self.softmax(self.cache['Z4'])
        return self.cache['A4']

    def _conv2d(self, x, W, b, stride=1, pad=1):
        N, C, H, W_in = x.shape
        F, _, HH, WW = W.shape  # F = number of filters

        # Add padding
        x_pad = np.pad(x, ((0,0), (0,0), (pad,pad), (pad,pad)), mode='constant')

        # Output dimensions
        H_out = (H + 2*pad - HH) // stride + 1
        W_out = (W_in + 2*pad - WW) // stride + 1

        out = np.zeros((N, F, H_out, W_out))

        for n in range(N):  # For each sample in batch
            for f in range(F):  # For each filter
                for i in range(H_out):  # Slide vertically
                    for j in range(W_out):  # Slide horizontally
                        ii, jj = i*stride, j*stride
                        # Element-wise multiplication and sum
                        out[n,f,i,j] = np.sum(
                            x_pad[n, :, ii:ii+HH, jj:jj+WW] * W[f]
                        ) + b[f]
        return out


    def _maxpool2d(self, x, pool_size=2, stride=2):
      N, C, H, W = x.shape
      H_out = (H - pool_size) // stride + 1
      W_out = (W - pool_size) // stride + 1

      out = np.zeros((N, C, H_out, W_out))

      for n in range(N):
          for c in range(C):
              for i in range(H_out):
                  for j in range(W_out):
                      ii, jj = i*stride, j*stride
                      out[n,c,i,j] = np.max(
                          x[n,c, ii:ii+pool_size, jj:jj+pool_size]
                      )
      return out

    def compute_loss(self, outputs, y_true):
        # Simplified loss using raw labels
        correct_probs = outputs[np.arange(len(y_true)), y_true]
        return -np.mean(np.log(correct_probs + 1e-10))  # Avoid log(0)

    def backward(self, x, y_true, lr=0.001):
        m = len(y_true)  # Batch size
        grads = {key: np.zeros_like(val) for key, val in self.params.items()}  # Initialize all gradients

        # Output layer gradient
        dZ4 = self.cache['A4'].copy()
        dZ4[np.arange(m), y_true] -= 1
        dZ4 /= m

        # FC layer gradients
        grads['W4'] = np.dot(self.cache['A3'].T, dZ4)
        grads['b4'] = np.sum(dZ4, axis=0)

        dA3 = np.dot(dZ4, self.params['W4'].T)
        dZ3 = dA3 * (self.cache['A3'] > 0)  # ReLU derivative

        # Hidden layer gradients
        grads['W3'] = np.dot(self.cache['F'].T, dZ3)
        grads['b3'] = np.sum(dZ3, axis=0)

        # Reshape for conv backprop
        dF = np.dot(dZ3, self.params['W3'].T)
        dP2 = dF.reshape(self.cache['P2'].shape)

        # MaxPool2 backward
        dA2 = np.zeros_like(self.cache['A2'])
        for n in range(dP2.shape[0]):
            for c in range(dP2.shape[1]):
                for i in range(dP2.shape[2]):
                    for j in range(dP2.shape[3]):
                        ii, jj = i*2, j*2
                        window = self.cache['A2'][n,c,ii:ii+2,jj:jj+2]
                        mask = (window == np.max(window))
                        dA2[n,c,ii:ii+2,jj:jj+2] = mask * dP2[n,c,i,j]

        # Conv2 backward
        dZ2 = dA2 * (self.cache['Z2'] > 0)

        # Calculate gradients for W2 and b2
        for n in range(x.shape[0]):
            for f in range(self.params['W2'].shape[0]):
                for c in range(self.params['W2'].shape[1]):
                    for i in range(dZ2.shape[2]):
                        for j in range(dZ2.shape[3]):
                            ii, jj = i, j  # No stride in gradient calculation
                            grads['W2'][f,c] += np.sum(
                                self.cache['P1'][n,c,ii:ii+3,jj:jj+3] * dZ2[n,f,i,j]
                            )
                grads['b2'][f] += np.sum(dZ2[n,f])

        # Backprop through Conv1
        dP1 = np.zeros_like(self.cache['P1'])  # Shape: (N, C, H, W)
        for n in range(x.shape[0]):  # Iterate over batch
            for c in range(self.params['W2'].shape[1]):  # Loop over input channels (not filters!)
                for f in range(self.params['W2'].shape[0]):  # Loop over filters
                    kernel = np.rot90(self.params['W2'][f, c], 2)  # Rotate the filter

                    # Add padding to dZ2 (same as Conv layer)
                    pad = 1  # Because Conv used a 3x3 filter with stride 1
                    dZ2_padded = np.pad(dZ2[n, f], ((pad, pad), (pad, pad)), mode='constant')

                    # Convolve manually
                    for i in range(dP1.shape[2]):  # Loop over height
                        for j in range(dP1.shape[3]):  # Loop over width
                            dP1[n, c, i, j] += np.sum(dZ2_padded[i:i+3, j:j+3] * kernel)  # Accumulate over filters


        # MaxPool1 backward
        dA1 = np.zeros_like(self.cache['A1'])
        for n in range(dP1.shape[0]):
            for c in range(dP1.shape[1]):
                for i in range(dP1.shape[2]):
                    for j in range(dP1.shape[3]):
                        ii, jj = i*2, j*2
                        window = self.cache['A1'][n,c,ii:ii+2,jj:jj+2]
                        mask = (window == np.max(window))
                        dA1[n,c,ii:ii+2,jj:jj+2] = mask * dP1[n,c,i,j]

        # Conv1 backward
        dZ1 = dA1 * (self.cache['Z1'] > 0)

        # Calculating gradients for W1 and b1
        for n in range(x.shape[0]):
            for f in range(self.params['W1'].shape[0]):
                for c in range(self.params['W1'].shape[1]):
                    for i in range(dZ1.shape[2]):
                        for j in range(dZ1.shape[3]):
                            ii, jj = i, j  # No stride in gradient calculation
                            grads['W1'][f,c] += np.sum(
                                x[n,c,ii:ii+3,jj:jj+3] * dZ1[n,f,i,j]
                            )
                grads['b1'][f] += np.sum(dZ1[n,f])

        # Normalising gradients by batch size
        for param in grads:
            grads[param] /= m

        # Updating parameters
        for param in self.params:
            self.params[param] -= lr * grads[param]

In [ ]:
# Initialise and train
num_classes = len(np.unique(y_labels))
cnn = ScratchCNN(input_shape=(1, height, width), num_classes=num_classes)

# Training loop
epochs = 10
batch_size = 32
losses = []

for epoch in range(epochs):
    epoch_loss = 0
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]

        # Forward pass
        outputs = cnn.forward(batch_X)

        # Loss calculation
        loss = cnn.compute_loss(outputs, batch_y)
        epoch_loss += loss

        # Backward pass
        cnn.backward(batch_X, batch_y, lr=0.001)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")
    losses.append(epoch_loss)

In [ ]:
# Evaluation
test_outputs = cnn.forward(X_test)
predicted_classes = np.argmax(test_outputs, axis=1)
accuracy = accuracy_score(y_test, predicted_classes)
print(f"\nTest Accuracy: {accuracy:.4f}")

# Plot training curve
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Progress")
plt.show()